In [2]:
import pandas as pd
import json
import sklearn
import glob
import pickle

pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
path='/mnt/swordfish-pool2/milad/hiatus-data/phase_2'

### Preparing data for explainability:

In [10]:
all_documents = []
for file in glob.glob(path + '/HRS2_test_document_records/english/*/*.jsonl'):
    print(file)
    all_documents+= [json.loads(x) for x in open(file).readlines()]

/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/foreground/HRS2.3_foreground_anonymized.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/foreground/HRS2.1_foreground_anonymized.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/foreground/HRS2.5_foreground_anonymized.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/foreground/HRS2.4_foreground_anonymized.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/foreground/HRS2.2_foreground_anonymized.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/background/HRS2.5_background_anonymized.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/background/HRS2.1_background_anonymized.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/HRS2_test_document_records/english/background/HRS2.2_back

In [4]:
with open(path+'/explainability/all_documents.jsonl', 'w') as output_file:
    for d in all_documents:
        json.dump(d, output_file)
        output_file.write('\n')

In [5]:
all_documents_df = pd.DataFrame(all_documents)
all_documents_df['authorID'] = all_documents_df.authorIDs.apply(lambda x:x[0])

In [22]:
all_documents_df.head()

,documentID,authorIDs,fullText,spanAttribution,isForeground,dateCollected,publiclyAvailable,collectionNum,source,deidentified,languages,lengthWords,dateCreated,timeCreated,sourceSpecific,authorID
0,3379c469-5fa9-5bc8-ab81-37c8ff266f45,[631b1b6d-5aac-5e18-8876-cc3085c9ab1a],Neutrogena Lathering Soap\n\nNeutrogena soap i...,[{'authorID': '631b1b6d-5aac-5e18-8876-cc3085c...,True,2024-02-06,False,HRS2.3,participant,True,[en],106,2024-02-06,22:17:39,"{'participantID': 'ER20001', 'internalDocID': 12}",631b1b6d-5aac-5e18-8876-cc3085c9ab1a
1,198e4e55-3ef9-50f7-b7f5-60f1f7952dd5,[631b1b6d-5aac-5e18-8876-cc3085c9ab1a],"Lululemon Align Tank Top, sonic pink, size 4\n...",[{'authorID': '631b1b6d-5aac-5e18-8876-cc3085c...,True,2024-02-06,False,HRS2.3,participant,True,[en],133,2024-02-06,22:17:40,"{'participantID': 'ER20001', 'internalDocID': 15}",631b1b6d-5aac-5e18-8876-cc3085c9ab1a
2,6c7bdb11-6021-56b1-b2cd-86d36639c9de,[631b1b6d-5aac-5e18-8876-cc3085c9ab1a],Retractable Double Dog Leash\n\nEverybody with...,[{'authorID': '631b1b6d-5aac-5e18-8876-cc3085c...,True,2024-02-06,False,HRS2.3,participant,True,[en],126,2024-02-06,22:17:40,"{'participantID': 'ER20001', 'internalDocID': 14}",631b1b6d-5aac-5e18-8876-cc3085c9ab1a
3,ab6c205a-7315-56ef-bcd5-116d9b0f3ec3,[631b1b6d-5aac-5e18-8876-cc3085c9ab1a],"Bic Erasable Pens, twelve pack, variety assort...",[{'authorID': '631b1b6d-5aac-5e18-8876-cc3085c...,True,2024-02-06,False,HRS2.3,participant,True,[en],125,2024-02-06,22:17:39,"{'participantID': 'ER20001', 'internalDocID': 13}",631b1b6d-5aac-5e18-8876-cc3085c9ab1a
4,1ffcebf2-70db-57cc-a366-f905427ed20c,[c41930b0-e8d8-57dd-9fc8-381c2275fcba],Tangle Teezer Detangling Hairbrush in Light Bl...,[{'authorID': 'c41930b0-e8d8-57dd-9fc8-381c227...,True,2024-03-01,False,HRS2.3,participant,True,[en],135,2024-03-01,15:50:15,"{'participantID': 'ER20022', 'internalDocID': ...",c41930b0-e8d8-57dd-9fc8-381c2275fcba


In [25]:
all_authors = all_documents_df["authorID"].unique()
training_atuhors, test_authors = sklearn.model_selection.train_test_split(all_authors, train_size=0.7, test_size=0.15, random_state=5, shuffle=True)

training_data = all_documents_df[all_documents_df['authorID'].isin(training_atuhors)]
test_data     = all_documents_df[all_documents_df['authorID'].isin(test_authors)]

In [26]:
training_data.to_json(path+'/explainability/train_authors.json')
test_data.to_json(path+'/explainability/test_authors.json')

#### Save documents that are appearing in cross genre data:

In [18]:
queries_and_cands = []
for file in glob.glob('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_crossGenre/TA2/hrs_06-27-24_english_crossGenre-combined/data/*.jsonl'):
    print(file)
    queries_and_cands+= [json.loads(x) for x in open(file).readlines()]

/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_crossGenre/TA2/hrs_06-27-24_english_crossGenre-combined/data/hrs_06-27-24_english_crossGenre-combined_TA2_input_candidates.jsonl
/mnt/swordfish-pool2/milad/hiatus-data/phase_2/mode_crossGenre/TA2/hrs_06-27-24_english_crossGenre-combined/data/hrs_06-27-24_english_crossGenre-combined_TA2_input_queries.jsonl


In [22]:
queries_and_cands_docs = [x['documentID'] for x in queries_and_cands]
all_data_docs =  [x['documentID'] for x in all_documents]

In [28]:
all_documents_in_cross_genre = [doc for doc in all_documents if doc['documentID'] in queries_and_cands_docs]

In [30]:
with open(path+'/explainability/all_documents_in_cross_genre.jsonl', 'w') as output_file:
    for d in all_documents_in_cross_genre:
        json.dump(d, output_file)
        output_file.write('\n')

### Generated Style Features:

In [16]:
df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/filtered/refined_and_aggregated_features_final.csv')

In [17]:
df.head()

,final_attribute_name,original_attribute_name,documentID,shortend_attribute_name.v1,shortend_attribute_name.v2,aggregated_name
0,simple sentence structures,"The author uses simple sentence structures, wi...",3379c469-5fa9-5bc8-ab81-37c8ff266f45,"The author uses simple sentence structures, wi...","The author uses simple sentence structures, wi...","The author uses simple sentence structures, wi..."
1,complex sentences,The author employs a mix of short and medium-l...,3379c469-5fa9-5bc8-ab81-37c8ff266f45,The author employs a mix of short and medium-l...,The author employs a mix of short and medium-l...,The author employs a mix of short and medium-l...
2,few instances,The author favors the use of present tense ver...,3379c469-5fa9-5bc8-ab81-37c8ff266f45,The author favors the use of present tense ver...,The author favors the use of present tense ver...,The author favors the use of present tense ver...
3,tense verbs,The author favors the use of present tense ver...,3379c469-5fa9-5bc8-ab81-37c8ff266f45,The author favors the use of present tense ver...,The author favors the use of present tense ver...,The author favors the use of present tense ver...
4,verb conjugation pattern,The author uses a consistent verb conjugation ...,3379c469-5fa9-5bc8-ab81-37c8ff266f45,The author uses a consistent verb conjugation ...,The author uses a consistent verb conjugation ...,The author uses a consistent verb conjugation ...


In [18]:
print(df.documentID.nunique())
print(df.final_attribute_name.nunique())

10513
216


In [14]:
df['documentID'].value_counts()

26120258-e5d2-56c5-9efb-ad3cb0d49e27    12
ce8ad4cf-7d58-5510-b90d-05123a67bf34    12
d06e01f5-a15b-5c6a-9806-5c4cb02daae2    12
a21523ad-853e-5c1c-a7ea-b3f18111575d    12
7c31ae33-c02e-5591-b3cc-1cd1e0a431d3    11
81a2cd1f-663e-5eba-8f7a-1ba0f95a5f52    11
a72dcae5-cc77-5265-a131-f56a1dca1125    11
2718f57a-cc37-5e39-acfa-213c654d2b5d    11
12ad8fc4-1d38-585d-9dc6-b972e96000af    11
f08cda44-4d2c-5585-a9d7-59acc93fec66    10
8a394173-d83b-5912-b6b3-1cca55079027    10
64a7685c-5c3e-5413-87f4-c1ce68788e6b    10
8a6c52f3-f1e8-5fb5-9b64-dd71e33340a6    10
10b12028-cef5-55fe-9236-e60fcbae30db    10
37e65d0d-1bec-5a14-9561-87af2f86a551    10
7c1fb803-c157-58a3-a704-b4768ad3e24b    10
3a17d2fa-7326-5717-979d-9c271ef6600c    10
c0816677-1707-5255-a9c5-0a053fbe3f97    10
47d60296-c31d-5df9-8f3a-7a75dbb42a0f    10
df895eb1-21e3-541f-8dcd-5dc2d7269da2    10
f74c2cf7-1b2f-502a-b7eb-687f36b0f98a    10
5cff5e67-a5e6-54ac-ba66-0eb4944bdee7    10
616d71fd-68f0-50dc-a327-f9b6c0455474    10
7a0033e8-4a

In [15]:
df['final_attribute_name'].value_counts()

long sentences                      3873
consistent verb tense               3308
simple sentence structures          2923
complex sentence structures         2376
past tense                          2327
proper nouns                        2322
formal tone                         2126
formal language                     2036
verb tenses                         1871
limited range                       1852
consistent use                      1847
complex sentences                   1671
informal language                   1409
declarative sentences               1373
average sentence length             1284
possessive pronouns                 1063
present tense                        939
subordinate clauses                  917
conversational tone                  855
passive voice                        764
short sentences                      505
moderate level                       473
colloquial expressions               367
interrogative sentences              352
relative pronoun

### Consturcted Interpretable space:

In [12]:
df = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/train_authors.pkl')

In [13]:
df.head()

,authorID,fullText,documentID,cluster_label,final_attribute_name
0,0047930d-c2d2-50d1-960a-ffdf702c36b9,Society of the Snow\n\nUGH! So fucking unfortu...,"[9d534839-55de-5d64-ab26-6037dfbd40ed, 371fa25...",-1,"[more narrative, informal language, long sente..."
1,00566bb1-243b-56fd-8282-04c6037d25f7,Perfect Days\n\nA simply Perfect film. He want...,"[ee9d1097-c053-53b8-b0d7-1c846a97e43f, f09118b...",-1,"[simple sentence structures, consistent verb t..."
2,006a1423-cc88-5428-9371-6e484f4308e4,Mama cat only feeding one kitten at a time\n\n...,[3da7f62d-f824-5910-86c3-8c7afa9b8701],-1,"[simple sentence structures, consistent verb t..."
3,00a796ed-a523-5229-ac6e-220994b70b46,Bravetco\n\nSo both my dogs had fleas a couple...,"[983df9f5-fbe3-5a01-bc04-adbac75468d0, a61f363...",-1,"[simple sentence structures, interrogative sen..."
4,00c8e55c-de51-5768-a1a7-4d888f49bb3b,Lightning Strikes Twice\n\nMuddled plotting an...,"[c4b78ec0-f466-534e-94a4-720b6e7d82da, d0143ff...",-1,"[simple sentence structures, consistent verb t..."


In [14]:
df.cluster_label.nunique()

24

In [15]:
ls /mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/

all_documents.jsonl                    test_authors.json
describe_documents_writing_styles.csv  train_authors.json
filtered/                              train_authors.pkl
interpretable_space.pkl                trainin_authors.json
style_corpus.csv


In [16]:
vector_to_style_dist = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/interpretable_space.pkl')

In [17]:
len(vector_to_style_dist)

24

In [91]:
[(key,sorted(x[1].items(), key=lambda i:-i[1])[:10]) for key, x in vector_to_style_dist.items()]

[(-1,
  [('simple sentence structures', 2433.283992983176),
   ('long sentences', 2236.8263367768163),
   ('consistent verb tense', 2184.1917810175187),
   ('limited range', 2118.3426694510413),
   ('past tense', 1945.3621859328655),
   ('declarative sentences', 1827.981784907458),
   ('informal language', 1730.3795565160267),
   ('formal language', 1584.1685958287278),
   ('average sentence length', 1537.0242707492507),
   ('present tense', 1478.3180604569516)]),
 (0,
  [('informal language', 26.12652059780296),
   ('conversational tone', 17.564864776473495),
   ('complex sentences', 11.035142021882624),
   ('vocabulary is diverse', 10.887468251263396),
   ('casual atmosphere', 10.69668972029537),
   ('different ideas', 10.69668972029537),
   ('casual tone', 10.259178624607298),
   ('colloquial expressions', 10.065018052563783),
   ('informal words', 8.55352248773499),
   ('conversational language', 8.47297468945911)]),
 (1,
  [('informal tone', 6.169325412217515),
   ('informal langu

### Generated Explanations:

In [73]:
test_df = pd.read_json('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/test_authors.json')

In [74]:
test_df[['documentID', 'documetn_style_description']].head()

,documentID,documetn_style_description
9,572eb600-05b3-52d5-8332-c8a30f336690,"[present tense, simple sentence structures, li..."
12,7a8bd5fe-5a9d-5d55-b207-2f8b9608ef8e,"[present tense, simple sentence structures, li..."
13,c4a2c366-5391-5376-a8a3-39f169b3998e,"[present tense, simple sentence structures, li..."
14,341e4705-31e6-5186-8601-be7ce7d1d89c,"[present tense, simple sentence structures, li..."
21,2901898e-c995-58ba-a5dc-ef6d9f510013,"[present tense, simple sentence structures, li..."
